# Spark SQL
---
Spark SQL is a Spark module for structured data processing. It provides a programming abstraction called DataFrames and can also act as distributed SQL query engine.

---
## DataFrames
A DataFrame is a distributed collection of data organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs.

The DataFrame API is available in Scala, Java, and Python.

All of the examples on this page use sample data included in the Spark distribution and can be run in the spark-shell or the pyspark shell.

In [1]:
# Import SQLContext and data types
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext

The entry point into all relational functionality in Spark is the SQLContext class, or one of its decedents. To create a basic SQLContext, all you need is a SparkContext.

In [2]:
conf = SparkConf()
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

## Creating DataFrames

Load a text file and convert each line to a tuple.
- 'file://' because it's local file

In [22]:
fname = 'file:///usr/local/spark/examples/src/main/resources/people.txt'
lines = sc.textFile(fname)

In [23]:
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))

### Infer the schema, and register the DataFrame as a table.

In [24]:
schemaPeople = sqlContext.inferSchema(people)
schemaPeople.registerTempTable("people")

/usr/local/spark/python/pyspark/sql/context.py:258: UserWarning: inferSchema is deprecated, please use createDataFrame instead.
  warnings.warn("inferSchema is deprecated, please use createDataFrame instead.")


#### SQL can be run over DataFrames that have been registered as a table.

In [26]:
teenagers = sqlContext.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")

The results of SQL queries are RDDs and support all the normal RDD operations.

In [14]:
teenNames = teenagers.map(lambda p: "Name: " + p.name)
for teenName in teenNames.collect():
    print(teenName)

Name: Justin


In [15]:
schemaPeople.cache()

DataFrame[age: bigint, name: string]

## DataFrame Operations
DataFrames provide a domain-specific language for structured data manipulation in Scala, Java, and Python.

Here some basic examples of structured data processing using DataFrames:

In [29]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [32]:
df = sqlContext.read.json("file:///usr/local/spark/examples/src/main/resources/people.json")

# Displays the content of the DataFrame to stdout
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [ ]:
df.printSchema()

In [ ]:
df.select("name").show()

In [ ]:
df.select("name", df.age + 1).show()

In [ ]:
df.filter(df.name > 21).show()

In [ ]:
df.groupBy("age").count().show()

In [ ]:
# help(df.agg)

## Programmatically Specifying the Schema

In [ ]:
# Import SQLContext and data types
from pyspark.sql import *
from pyspark.sql.types import *

In [ ]:
# sc is an existing SparkContext.
sqlContext = SQLContext(sc)

#### Load a text file and convert each line to a tuple.

In [ ]:
fname = "file:///usr/local/spark/examples/src/main/resources/people.txt"
lines = sc.textFile(fname)
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: (p[0], p[1].strip()))

#### The schema is encoded in a string.

In [ ]:
schemaString = "name age"
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

#### Apply the schema to the RDD.

In [ ]:
schemaPeople = sqlContext.createDataFrame(people, schema)

#### Register the DataFrame as a table.

In [ ]:
schemaPeople.registerTempTable("people")

#### SQL can be run over DataFrames that have been registered as a table.

In [ ]:
results = sqlContext.sql("SELECT name FROM people")

#### The results of SQL queries are RDDs and support all the normal RDD operations.

In [ ]:
names = results.map(lambda p: "Name: " + p.name)
for name in names.collect():
    print(name)

In [27]:
teenagers

+------+
|  name|
+------+
|Justin|
+------+

